In [1]:
import pandas as pd

business_class= pd.read_csv('D:/IDMP/Final_Project/FlightPricePrediction/data/business.csv')
economy_class = pd.read_csv('D:/IDMP/Final_Project/FlightPricePrediction/data/economy.csv')

business_class['class']='Business'
economy_class['class']='Economy'

print(business_class)
print(economy_class)

             date    airline ch_code  num_code dep_time     from time_taken  \
0      11-02-2022  Air India      AI       868    18:00    Delhi    02h 00m   
1      11-02-2022  Air India      AI       624    19:00    Delhi    02h 15m   
2      11-02-2022  Air India      AI       531    20:00    Delhi    24h 45m   
3      11-02-2022  Air India      AI       839    21:25    Delhi    26h 30m   
4      11-02-2022  Air India      AI       544    17:15    Delhi    06h 40m   
...           ...        ...     ...       ...      ...      ...        ...   
93482  31-03-2022    Vistara      UK       822    09:45  Chennai    10h 05m   
93483  31-03-2022    Vistara      UK       826    12:30  Chennai    10h 25m   
93484  31-03-2022    Vistara      UK       832    07:05  Chennai    13h 50m   
93485  31-03-2022    Vistara      UK       828    07:00  Chennai    10h 00m   
93486  31-03-2022    Vistara      UK       822    09:45  Chennai    10h 05m   

                                                   

In [2]:
flight_data = pd.concat([business_class, economy_class])


In [3]:
# Reset the index
flight_data = flight_data.reset_index(drop=True)
flight_data

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",Business
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",Business
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220",Business
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450",Business
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690",Business
...,...,...,...,...,...,...,...,...,...,...,...,...
300256,31-03-2022,Vistara,UK,832,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697",Economy
300257,31-03-2022,Vistara,UK,832,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709",Economy
300258,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy
300259,31-03-2022,Vistara,UK,822,9:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy


In [4]:
flight_data['flight_number'] = flight_data['ch_code'] + '-' + flight_data['num_code'].astype(str)


In [5]:
flight_data

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,class,flight_number
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",Business,AI-868
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",Business,AI-624
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220",Business,AI-531
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450",Business,AI-839
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690",Business,AI-544
...,...,...,...,...,...,...,...,...,...,...,...,...,...
300256,31-03-2022,Vistara,UK,832,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697",Economy,UK-832
300257,31-03-2022,Vistara,UK,832,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709",Economy,UK-832
300258,31-03-2022,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy,UK-826
300259,31-03-2022,Vistara,UK,822,9:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy,UK-822


In [6]:
flight_data = flight_data.drop(['ch_code', 'num_code'], axis=1)


In [7]:
flight_data

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,class,flight_number
0,11-02-2022,Air India,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612",Business,AI-868
1,11-02-2022,Air India,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612",Business,AI-624
2,11-02-2022,Air India,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220",Business,AI-531
3,11-02-2022,Air India,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450",Business,AI-839
4,11-02-2022,Air India,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690",Business,AI-544
...,...,...,...,...,...,...,...,...,...,...,...
300256,31-03-2022,Vistara,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,697",Economy,UK-832
300257,31-03-2022,Vistara,7:05,Chennai,13h 50m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:55,Hyderabad,"7,709",Economy,UK-832
300258,31-03-2022,Vistara,12:30,Chennai,20h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy,UK-826
300259,31-03-2022,Vistara,9:45,Chennai,23h 20m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,9:05,Hyderabad,"8,640",Economy,UK-822


In [8]:
print(flight_data.isnull().sum().sum())


0


In [9]:
print(flight_data['stop'].unique())


['non-stop ' '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXU\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Chennai\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Lucknow\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia STV\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Hyderabad\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia GAY\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '2+-stop'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Guwahati\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia GAU\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia VTZ\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia NDC\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXE\n\t\t\t\t\t\t\t\t\t\t\t\t'
 '1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Raipur\n\t\t\t\t\t\t\t\t\t\t\t

In [10]:
flight_data['stop'] = flight_data['stop'].replace(r'[\n\t]+', ' ', regex=True)


In [11]:
flight_data['stop'] = flight_data['stop'].apply(lambda x: x.split(' Via ')[0])


In [12]:
mapping_dict = {'non-stop ': 'zero',
                '1-stop': 'one',
                '1-stop ': 'one',
                '2+-stop': 'two_or_more'}

flight_data['stop'] = flight_data['stop'].replace(mapping_dict)

In [13]:
flight_data

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,class,flight_number
0,11-02-2022,Air India,18:00,Delhi,02h 00m,zero,20:00,Mumbai,"25,612",Business,AI-868
1,11-02-2022,Air India,19:00,Delhi,02h 15m,zero,21:15,Mumbai,"25,612",Business,AI-624
2,11-02-2022,Air India,20:00,Delhi,24h 45m,one,20:45,Mumbai,"42,220",Business,AI-531
3,11-02-2022,Air India,21:25,Delhi,26h 30m,one,23:55,Mumbai,"44,450",Business,AI-839
4,11-02-2022,Air India,17:15,Delhi,06h 40m,one,23:55,Mumbai,"46,690",Business,AI-544
...,...,...,...,...,...,...,...,...,...,...,...
300256,31-03-2022,Vistara,7:05,Chennai,13h 50m,one,20:55,Hyderabad,"7,697",Economy,UK-832
300257,31-03-2022,Vistara,7:05,Chennai,13h 50m,one,20:55,Hyderabad,"7,709",Economy,UK-832
300258,31-03-2022,Vistara,12:30,Chennai,20h 35m,one,9:05,Hyderabad,"8,640",Economy,UK-826
300259,31-03-2022,Vistara,9:45,Chennai,23h 20m,one,9:05,Hyderabad,"8,640",Economy,UK-822


In [14]:
from datetime import datetime
flight_data['date'] = flight_data['date'].replace(r'/', '-', regex=True)
flight_data['date'] = pd.to_datetime(flight_data['date'], format='%d-%m-%Y')

particular_date = datetime.strptime('10-02-2022', '%d-%m-%Y')

flight_data['days_left'] = (flight_data['date'] - particular_date).dt.days

print(flight_data)


             date    airline dep_time     from time_taken  stop arr_time  \
0      2022-02-11  Air India    18:00    Delhi    02h 00m  zero    20:00   
1      2022-02-11  Air India    19:00    Delhi    02h 15m  zero    21:15   
2      2022-02-11  Air India    20:00    Delhi    24h 45m   one    20:45   
3      2022-02-11  Air India    21:25    Delhi    26h 30m   one    23:55   
4      2022-02-11  Air India    17:15    Delhi    06h 40m   one    23:55   
...           ...        ...      ...      ...        ...   ...      ...   
300256 2022-03-31    Vistara     7:05  Chennai    13h 50m   one    20:55   
300257 2022-03-31    Vistara     7:05  Chennai    13h 50m   one    20:55   
300258 2022-03-31    Vistara    12:30  Chennai    20h 35m   one     9:05   
300259 2022-03-31    Vistara     9:45  Chennai    23h 20m   one     9:05   
300260 2022-03-31    Vistara    20:30  Chennai    24h 25m   one    20:55   

               to   price     class flight_number  days_left  
0          Mumbai  25,61

In [15]:
flight_data

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,class,flight_number,days_left
0,2022-02-11,Air India,18:00,Delhi,02h 00m,zero,20:00,Mumbai,"25,612",Business,AI-868,1
1,2022-02-11,Air India,19:00,Delhi,02h 15m,zero,21:15,Mumbai,"25,612",Business,AI-624,1
2,2022-02-11,Air India,20:00,Delhi,24h 45m,one,20:45,Mumbai,"42,220",Business,AI-531,1
3,2022-02-11,Air India,21:25,Delhi,26h 30m,one,23:55,Mumbai,"44,450",Business,AI-839,1
4,2022-02-11,Air India,17:15,Delhi,06h 40m,one,23:55,Mumbai,"46,690",Business,AI-544,1
...,...,...,...,...,...,...,...,...,...,...,...,...
300256,2022-03-31,Vistara,7:05,Chennai,13h 50m,one,20:55,Hyderabad,"7,697",Economy,UK-832,49
300257,2022-03-31,Vistara,7:05,Chennai,13h 50m,one,20:55,Hyderabad,"7,709",Economy,UK-832,49
300258,2022-03-31,Vistara,12:30,Chennai,20h 35m,one,9:05,Hyderabad,"8,640",Economy,UK-826,49
300259,2022-03-31,Vistara,9:45,Chennai,23h 20m,one,9:05,Hyderabad,"8,640",Economy,UK-822,49


In [16]:
def get_time_category(dep_time):
    dep_hour = int(dep_time.split(':')[0])
    
    if dep_hour >= 4 and dep_hour < 6:
        return 'Early_Morning'
    elif dep_hour >= 6 and dep_hour < 12:
        return 'Morning'
    elif dep_hour >= 12 and dep_hour < 16:
        return 'Afternoon'
    elif dep_hour >= 16 and dep_hour < 22:
        return 'Evening'
    else:
        return 'Night'

In [17]:
flight_data['dep_time'] = flight_data['dep_time'].apply(get_time_category)
flight_data['arr_time'] = flight_data['arr_time'].apply(get_time_category)
flight_data

,date,airline,dep_time,from,time_taken,stop,arr_time,to,price,class,flight_number,days_left
0,2022-02-11,Air India,Evening,Delhi,02h 00m,zero,Evening,Mumbai,"25,612",Business,AI-868,1
1,2022-02-11,Air India,Evening,Delhi,02h 15m,zero,Evening,Mumbai,"25,612",Business,AI-624,1
2,2022-02-11,Air India,Evening,Delhi,24h 45m,one,Evening,Mumbai,"42,220",Business,AI-531,1
3,2022-02-11,Air India,Evening,Delhi,26h 30m,one,Night,Mumbai,"44,450",Business,AI-839,1
4,2022-02-11,Air India,Evening,Delhi,06h 40m,one,Night,Mumbai,"46,690",Business,AI-544,1
...,...,...,...,...,...,...,...,...,...,...,...,...
300256,2022-03-31,Vistara,Morning,Chennai,13h 50m,one,Evening,Hyderabad,"7,697",Economy,UK-832,49
300257,2022-03-31,Vistara,Morning,Chennai,13h 50m,one,Evening,Hyderabad,"7,709",Economy,UK-832,49
300258,2022-03-31,Vistara,Afternoon,Chennai,20h 35m,one,Morning,Hyderabad,"8,640",Economy,UK-826,49
300259,2022-03-31,Vistara,Morning,Chennai,23h 20m,one,Morning,Hyderabad,"8,640",Economy,UK-822,49


In [18]:
flight_data[['hours', 'minutes']] = flight_data['time_taken'].str.extract('(\d+)h\s*(\d+)m')
flight_data['hours'] = pd.to_numeric(flight_data['hours'])
flight_data['minutes'] = pd.to_numeric(flight_data['minutes'])
flight_data['duration'] = flight_data['hours'] + flight_data['minutes']/60
flight_data = flight_data.drop(columns=['hours', 'minutes','time_taken'])

flight_data

,date,airline,dep_time,from,stop,arr_time,to,price,class,flight_number,days_left,duration
0,2022-02-11,Air India,Evening,Delhi,zero,Evening,Mumbai,"25,612",Business,AI-868,1,2.000000
1,2022-02-11,Air India,Evening,Delhi,zero,Evening,Mumbai,"25,612",Business,AI-624,1,2.250000
2,2022-02-11,Air India,Evening,Delhi,one,Evening,Mumbai,"42,220",Business,AI-531,1,24.750000
3,2022-02-11,Air India,Evening,Delhi,one,Night,Mumbai,"44,450",Business,AI-839,1,26.500000
4,2022-02-11,Air India,Evening,Delhi,one,Night,Mumbai,"46,690",Business,AI-544,1,6.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
300256,2022-03-31,Vistara,Morning,Chennai,one,Evening,Hyderabad,"7,697",Economy,UK-832,49,13.833333
300257,2022-03-31,Vistara,Morning,Chennai,one,Evening,Hyderabad,"7,709",Economy,UK-832,49,13.833333
300258,2022-03-31,Vistara,Afternoon,Chennai,one,Morning,Hyderabad,"8,640",Economy,UK-826,49,20.583333
300259,2022-03-31,Vistara,Morning,Chennai,one,Morning,Hyderabad,"8,640",Economy,UK-822,49,23.333333
